In [109]:
import ctf,random
import numpy as np
import numpy.linalg as la
from ctf import random as crandom
glob_comm = ctf.comm()

In [110]:
#TODO

#def updateU(T,V,W,regParam,omega,I,J,K,r):
#    '''Update U matrix by using the formula'''
#    
#    M1 = ctf.tensor((J,K,r))
#    M1.i("jku") << V.i("ju")*W.i("ku")
#    [U_,S_,V_] = ctf.svd(M1.reshape((J*K,r)))
#    S_ = 1./S_
#    U.set_zero()
#    U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
#    normalize(U)
#    
#    return U   
#    
#    
#def updateV(T,U,W,regParam,omega,I,J,K,r):
#    '''Update V matrix by using the formula'''
#    
#    M2 = ctf.tensor((I,K,r))
#    M2.i("iku") << U.i("iu")*W.i("ku")
#    [U_,S_,V_] = ctf.svd(M2.reshape((I*K,r)))
#    S_ = 1./S_
#    V.set_zero()
#    V.i("ju") << V_.i("vu")*S_.i("v")*U_.reshape((I,K,r)).i("ikv")*T.i("ijk")
#    normalize(V)
#    
#    return V   
#
#def updateW(T,U,V,regParam,omega,I,J,K,r):
#    '''Update V matrix by using the formula'''
#    
#    M3 = ctf.tensor((I,J,r))
#    M3.i("iju") << U.i("iu")*V.i("ju")
#    [U_,S_,V_] = ctf.svd(M3.reshape((I*J,r)))
#    S_ = 1./S_
#    W.set_zero()
#    W.i("ku") << V_.i("vu")*S_.i("v")*U_.reshape((I,J,r)).i("ijv")*T.i("ijk")
#    normalize(W)
#    
#    return W

In [111]:
#def getALSCtf(T,U,V,W,regParam,omega,I,J,K,r):
#    '''
#    Same thing as above, but CTF
#    '''
#    it = 0
#    E = ctf.tensor((I,J,K))
#    E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
#    curr_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
#    
#    while True:
#        U = updateU(T,V,W,regParam,omega,I,J,K,r) 
#        V = updateV(T,U,W,regParam,omega,I,J,K,r) 
#        W = updateW(T,U,V,regParam,omega,I,J,K,r) 
#        E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
#        next_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
#        
#        if curr_err_norm - next_err_norm > .001 or it > 10:
#            break
#            
#        print(curr_err_norm, next_err_norm)
#        curr_err_norm = next_err_norm
#        it += 1
#        
#    return U,V,W

In [112]:
def normalize(Z):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

In [113]:
def init_tensor(n):
    dimensions = []
    while n>0:
        r = random.randint(1,10)
        dimensions.append(r)
        n -= 1
    #print("dimensions: ", tuple(dimensions))
    T = ctf.tensor(tuple(dimensions),sp=True)
    T.fill_sp_random(0,1,1)
    return T,dimensions


def init_factors(dimensions,r):
    D = dimensions.copy()
    ctf.random.seed(42)
    factors = []
    while len(D) != 0:
        I = D.pop(0)
        U = ctf.random.random((I,r))
        normalize(U)
        factors.append(U)
    return factors

def updateOmega(dimensions,sparsity):
    '''
    Gets a random subset of rows
    '''
    Actf = ctf.tensor(tuple(dimensions),sp=True)
    Actf.fill_sp_random(0,1,sparsity)
    omegactf = ((Actf > 0)*ctf.astensor(1.))
    return omegactf

In [116]:
n = random.randint(1,5) # arbitrary dimension
r = n # TODO: determine rank?
sparsity = .2
regParam = 2
  
# arbitrary tensor
T,dimensions = init_tensor(n)
print("dimensions: ", tuple(dimensions))

factors = init_factors(dimensions,r)

omega = updateOmega(dimensions,sparsity)

dimensions:  (10, 2, 8, 4, 9)
